In [1]:
%cd src
import random
import json
import os
import re
import subprocess
import multiprocessing
from collections import defaultdict
from accelerate import Accelerator
import json
import random
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from refinement import *
from generator.conv import GenerateConversation
from generator.rec import GenerateRecommendation
from generator.hooks import hook_sentences ,promotional_sentences, pick_verbs
from fuzzywuzzy import fuzz
multiprocessing.set_start_method('spawn', force=True)

/home/thiendc/InferConverRec/src


In [2]:
def get_entities_and_ids(json_file_path):
    with open(json_file_path, 'r') as json_file:
        data = json.load(json_file)

    entity_id = {}
    id_entity = {}
    for k, v in data.items():
        k = k.split("/")[-1]
        k = k.replace(">", "")
        # if "(" and ")" in k:
        #     k=re.sub("[\(\[].*?[\)\]]", "", k)
        k = k.replace("_", " ")
        k = k.strip()
        # k = k.lower()

        entity_id[k] = v
        id_entity[v] = k

    return entity_id

def cosine_similarity_entities(entities):
    matched_entities = []
    entity_names = list(get_entities_and_ids('./data/redial_gen/entity2id.json').keys())
    
    # Tạo vector TF-IDF cho tất cả thực thể trong entity_names và entities
    vectorizer = TfidfVectorizer().fit(entity_names + entities)
    
    # Vector hóa danh sách entity
    entity_vectors = vectorizer.transform(entities)
    entity_name_vectors = vectorizer.transform(entity_names)

    # Duyệt qua từng thực thể
    for entity_vector in entity_vectors:
        # Tính cosine similarity giữa entity hiện tại và tất cả entity_names
        similarities = cosine_similarity(entity_vector, entity_name_vectors).flatten()

        # Sắp xếp theo độ tương đồng giảm dần
        sorted_similarities = sorted(zip(entity_names, similarities), key=lambda x: x[1], reverse=True)

        # Lấy top 10 kết quả có độ tương đồng > 0.5 (50%)
        top_matches = [entity_name for entity_name, similarity in sorted_similarities[:10] if similarity > 0.5]
        matched_entities.extend(top_matches)

    return matched_entities


In [3]:
def input2jsonl(input_str, entity2id_json):
    save_path = './data/redial/sample_input_data_processed.jsonl'
    is_exist = os.path.exists(save_path)

    current_str = ''

    if is_exist:
        with open(save_path, 'r') as json_file:
            current_str = json.load(json_file)
            current_str['context'].append(input_str)
    else:
        current_str = {"context": [input_str], "resp": "", "rec": [], "entity": []}
    
    entities = re.findall(r'\$.*?\$', input_str)
    if entities:
        entities = [entity.replace('$', '').lower() for entity in entities]
        # list_entities = set(get_entities_and_ids(entity2id_json))

        matched_entities = cosine_similarity_entities(entities)
        entity_id = get_entities_and_ids('./data/redial_gen/entity2id.json')
        if matched_entities:

            entity_ids = [entity_id[entity] for entity in matched_entities if entity in get_entities_and_ids(entity2id_json)]
            print("-------------------------------------------------------------------")
            print(f"Movies that you {random.choice(pick_verbs)} earlier: {', '.join(matched_entities)}")
            print(f"Corresponding IDs : {', '.join(map(str, entity_ids))}")
            # print("-------------------------------------------------------------------")
            current_str['rec'].extend(entity_ids)

            does_movie_exist = True
        else:
            does_movie_exist = False
    else:
        does_movie_exist = False
    
    with open(save_path, 'w') as outfile:
        jout = json.dumps(current_str)
        outfile.write(jout)
    
    return does_movie_exist

In [4]:
global_conv_module = None

def initialize_conv_module():
    global global_conv_module
    if global_conv_module is None:
        global_conv_module = GenerateConversation()
initialize_conv_module()
global_conv_module.generate_conversations()

/home/thiendc/projects/.conda/lib/python3.11/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['edge_index', 'edge_type'] []


  0%|          | 0/1 [00:00<?, ?it/s]

/home/thiendc/projects/.conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/thiendc/projects/.conda/lib/python3.11/site-packages/accelerate/accelerator.py:604: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


  0%|          | 0/1 [00:04<?, ?it/s]

...

[2024-09-24 02:39:54,313] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/thiendc/projects/.conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/home/thiendc/projects/.conda/compiler_compat/ld: cannot find -lcufile: No such file or directory
collect2: error: ld returned 1 exit status
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


-----------------------------------
Generated response: ['System: Yes, it is. What kind of movies do you like? Have a good day! You are welcome and have fun with the guy who did <movie>. It was really funny how about <movie>? That one has an actor named Michael Caine as well if anyone wants more comedy they should check out that film called The Great Detective<|endoftext|>']
-----------------------------------


[W924 02:39:57.046159946 CudaIPCTypes.cpp:16] Producer process has been terminated before all shared CUDA tensors released. See Note [Sharing CUDA tensors]


'System: Yes, it is. What kind of movies do you like? Have a good day! You are welcome and have fun with the guy who did <movie>. It was really funny how about <movie>? That one has an actor named Michael Caine as well if anyone wants more comedy they should check out that film called The Great Detective<|endoftext|>'

In [ ]:
input_str =  input()
global global_conv_module
doesEntityExist = input2jsonl(input_str, entity2id_json = './data/redial_gen/entity2id.json')    

print("***************")
print("Does entity exist:", doesEntityExist)
print("***************")

if doesEntityExist:
    # Initialize conv_module if it hasn't been initialized yet
    initialize_conv_module()
    # Use the global conv_module
    predict_conversation = global_conv_module.generate_conversations()
    print('Đây là conversation', predict_conversation) # list

    save_path = './data/redial/sample_input_data_processed.jsonl'
    try:
        with open(save_path, 'r') as json_file:
            data = json.load(json_file)
    except json.JSONDecodeError:
        # If the file is empty or invalid JSON, create a new dictionary
        data = {"context": [], "resp": "", "rec": [], "entity": []}

    if isinstance(predict_conversation, list):
        predict_conversation = ' '.join(predict_conversation)

    predict_conversation = rewrite2(predict_conversation)
    predict_conversation = predict_conversation.replace("<|endoftext|>", "")
    predict_conversation = predict_conversation.replace("System: ", "")
    
    if not any(q in predict_conversation for q in question_type):
        if any(word in ['yes', 'no'] for word in predict_conversation.split(".")):
            predict_conversation = ".".join([i for i in predict_conversation.split(".") if 'yes' not in i and 'no' not in i])

    if predict_conversation.strip() == "":
        predict_conversation = random.choice(hook_sentences)
    
    if is_valid_sentence(predict_conversation):
        predict_conversation = rewrite(predict_conversation)

    data['resp'] = predict_conversation

    with open(save_path, 'w') as json_file:
        json.dump(data, json_file)

    
    subprocess.run([
        "accelerate", "launch", 
        "--num_processes", "2",  # Chính xác 4 processes cho 4 GPU
        "infer_rec.py",
        "--dataset", "redial_gen",
        "--tokenizer", "/home/thiendc/InferConverRec/src/utils/dialogpt",
        "--model", "/home/thiendc/InferConverRec/src/utils/dialogpt_model",
        "--text_tokenizer", "/home/thiendc/InferConverRec/src/utils/roberta",
        "--text_encoder", "/home/thiendc/InferConverRec/src/utils/roberta_model",
        "--n_prefix_rec", "20",
        "--prompt_encoder", "/home/thiendc/InferConverRec/src/output_dir/rec1/best",
        "--per_device_eval_batch_size", "16",
        "--gradient_accumulation_steps", "2",
        "--context_max_length", "128",
        "--prompt_max_length", "128",
        "--entity_max_length", "32",
    ], check=True)

    
    

-------------------------------------------------------------------
Movies that you went with earlier: spider-man 3, spider-man 2, spider-man, spider, the amazing spider-man, the amazing spider-man 2, spider-man: homecoming, spider-man strikes back, 9-man
Corresponding IDs : 1527, 4771, 22449, 20442, 7525, 29481, 5137, 17787, 28225
***************
Does entity exist: True
***************


/home/thiendc/projects/.conda/lib/python3.11/site-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


['edge_index', 'edge_type'] []


  0%|          | 0/1 [00:00<?, ?it/s]

/home/thiendc/projects/.conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
/home/thiendc/projects/.conda/lib/python3.11/site-packages/accelerate/accelerator.py:604: FutureWarning: The `use_fp16` property is deprecated and will be removed in version 1.0 of Accelerate use `Accelerator.mixed_precision == 'fp16'` instead.
  warnings.warn(
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/home/thiendc/projects/.conda/lib/python3.11/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/thiendc/projects/.conda/lib/python3.11/mult

KeyboardInterrupt: 

In [21]:
with open('./data/redial_gen/movie_ids.json', 'r') as moviesl:
    list_movies = json.load(moviesl)

with open('./recommedations.json', 'r') as f:
    movie_rec = json.load(f)

recommendation_counts = defaultdict(int)

# Duyệt qua từng item trong movie_rec và đếm số lần xuất hiện của mỗi recommendation
for item in movie_rec:
    recommendations = item[0]['recommendation']
    for rec in recommendations:
        recommendation_counts[rec] += 1

# Sắp xếp các recommendation dựa trên số lần xuất hiện (tần suất) và tạo list unique gt
sorted_recommendations = sorted(recommendation_counts.keys(), key=lambda x: recommendation_counts[x], reverse=True)

# Nhóm các unique 'gt' vào list
unique_gt = list({item[0]['gt'] for item in movie_rec})

# In kết quả
print("Unique GTs:", unique_gt)
print("Ranked Recommendations:", sorted_recommendations)


{'gt': 5831, 'recommendation': [2796, 7246, 29492, 12603, 24963, 5137, 15145, 20216, 14653, 20490, 20212, 28721, 20444, 6401, 4118, 16056, 20149, 7683, 16740, 22449, 20827, 27093, 10573, 11803, 7701, 1738, 7692, 9636, 13425, 372, 15370, 20394, 15063, 19346, 20266, 9457, 16847, 28647, 30737, 12753, 2547, 28693, 28205, 1113, 6604, 17098, 16688, 3429, 8061, 9277]}
{'gt': 29492, 'recommendation': [2796, 7246, 29492, 12603, 24963, 5137, 15145, 20216, 14653, 20490, 20212, 28721, 20444, 6401, 4118, 16056, 20149, 7683, 16740, 22449, 20827, 27093, 10573, 11803, 7701, 1738, 7692, 9636, 13425, 372, 15370, 20394, 15063, 19346, 20266, 9457, 16847, 28647, 30737, 12753, 2547, 28693, 28205, 1113, 6604, 17098, 16688, 3429, 8061, 9277]}
{'gt': 2216, 'recommendation': [2796, 7246, 29492, 12603, 24963, 5137, 15145, 20216, 14653, 20490, 20212, 28721, 20444, 6401, 4118, 16056, 20149, 7683, 16740, 22449, 20827, 27093, 10573, 11803, 7701, 1738, 7692, 9636, 13425, 372, 15370, 20394, 15063, 19346, 20266, 9457, 

In [22]:
from collections import defaultdict

# Tạo cấu trúc defaultdict để lưu số lần xuất hiện của các recommendation
recommendation_counts = defaultdict(int)

# Duyệt qua từng item trong movie_rec và đếm số lần xuất hiện của mỗi recommendation
for item in movie_rec:
    recommendations = item[0]['recommendation']
    for rec in recommendations:
        recommendation_counts[rec] += 1

# Sắp xếp các recommendation dựa trên số lần xuất hiện (tần suất) và tạo list unique gt
sorted_recommendations = sorted(recommendation_counts.keys(), key=lambda x: recommendation_counts[x], reverse=True)
# Nhóm các unique 'gt' vào list
unique_gt = list({item[0]['gt'] for item in movie_rec})

# In kết quả
print("Unique GTs:", unique_gt)
print("Ranked Recommendations:", sorted_recommendations)

if predict_conversation.count("<movie>") >= 1:
    data['rec'] = sorted_recommendations[:predict_conversation.count("<movie>")]
else:
    data['rec'] = sorted_recommendations[:3]

response = data['resp']
recommended_movies = data['rec']

if "<movie>" in response:
    for movie_id in recommended_movies:
        

Unique GTs: [2216, 29492, 5831]
Ranked Recommendations: [2796, 7246, 29492, 12603, 24963, 5137, 15145, 20216, 14653, 20490, 20212, 28721, 20444, 6401, 4118, 16056, 20149, 7683, 16740, 22449, 20827, 27093, 10573, 11803, 7701, 1738, 7692, 9636, 13425, 372, 15370, 20394, 15063, 19346, 20266, 16847, 28647, 30737, 12753, 2547, 28693, 28205, 1113, 6604, 17098, 9277, 9457, 16688, 3429, 8061, 29510, 28908, 7423, 11048]


In [ ]:
with open('./data/redial_gen/movie_ids.json', 'r') as moviesl:
    list_movies = json.load(moviesl)
with open('./recommedations.json', 'r') as f:
    movie_rec = json.load(f)
movie_rec = movie_rec[-1][0]
data['rec'] = [i for i in movie_rec['recommendation'] if i in list_movies][:predict_conversation.count("<movie>")]
response = data['resp']
recommended_movies = data['rec']

with open('/home/thiendc/InferConverRec/src/data/redial/entity2id.json', 'r') as f:
    movielink2id =  json.load(f)
id2movielink = {v: k for k, v in movielink2id.items()}

for movie_id in recommended_movies:
    if '<movie>' in response:
        if isinstance(movie_id, list):
            movie_id = movie_id[0]  # Take the first item if it's a list
        movie_link = id2movielink.get(movie_id, str(movie_id))  # Use get() with a default value
        movie_name = movie_link.rstrip("/").split('/')[-1].strip('>').replace("_", " ")
        response = response.replace("<movie>", movie_name, 1)  # Replace only one occurrence
    else:
        response = response + " " + random.choice(promotional_sentences)
        movie_link = id2movielink.get(movie_id, str(movie_id))
        movie_name = movie_link.rstrip("/").split('/')[-1].strip('>').replace("_", " ")
        response = response + " " + movie_name + "."

with open('/home/thiendc/InferConverRec/src/data/redial/sample_input_data_processed.jsonl', 'r') as file:
    line = file.readline().strip()
data = json.loads(line)
data['context'].append(response)